# 1 Join text and image representation 

In [ ]:
import tensorflow from tensorflow.keras.models
import Sequential, load_model
import numpy as np from tensorflow.keras.applications.resnet50
import ResNet50 from tensorflow.keras.applications.resnet50 
import preprocess_input,␣ ‹→decode_predictions from tensorflow.keras.preprocessing
import image from tensorflow.keras.applications.imagenet_utils 
import preprocess_input  resnet_model = ResNet50(weights='imagenet' , include_top= False) 

In [ ]:
def extract_features(img_path):
    img = image.load_img(img_path, target_size=(224, 224)) x =
    image.img_to_array(img)
    x = np.expand_dims(x, axis=0) x =
    preprocess_input(x) features = resnet_model.predict(x)
    return np.expand_dims(features.flatten(), axis=0) 

# 2 Image representation

In [ ]:
features = np.load('resnet50-features.10k.npy')
print(features.shape) 

# 3 Text Representation 

In [ ]:
def load(captions_filename, features_filename):
    features = np.load(features_filename) images = [] texts = []
    with open(captions_filename) as fp: 
        for line in fp: tokens = line.strip().split()
            images.append(tokens[0]) texts.append(' '.join(tokens[1:]))
            return features, images, texts 

In [ ]:
feature , images , texts = load( "annotations.10k.txt", "resnet50-features.10k. ‹→npy") 

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts) captions = pad_sequences(sequences, maxlen=16) 

In [ ]:
vocab = tokenizer.word_index
vocab['<eos>'] = 0 

In [ ]:
import json with open('vocab.json', 'w') as fp: # save the vocab
    fp.write(json.dumps(vocab)) 

# 3.1 load the pretrained embedded model

In [ ]:
import embedding 
embedding_weights = embedding.load(vocab, 100,'glove.twitter.27B.100d.filtered. ‹→txt')

# 4 putting together the model 

In [ ]:
from keras.layers import Input, Dense, Embedding, GRU
image_input = Input(shape=(2048,))
caption_input = Input(shape=(16,))
noise_input = Input(shape=(16,))
caption_embedding = Embedding(len(vocab), 100,)
input_length=16,weights=[embedding_weights]
caption_rnn = GRU(256)
image_dense = Dense(256, activation='tanh') 

# 4.1 creation of pipe line 

In [ ]:
image_pipeline = image_dense(image_input)
caption_pipeline = caption_rnn(caption_embedding(caption_input))
noise_pipeline = caption_rnn(caption_embedding(noise_input)) 

# 4.2 compute the dot product between image and caption 

In [ ]:
from keras.layers import merge positive_pair = merge.dot([image_pipeline, caption_pipeline] , axes=1,normalize=False ) 
negative_pair = merge.dot([image_pipeline, noise_pipeline], axes=1,normalize=False)
output = merge.concatenate([positive_pair, negative_pair] ) 

In [ ]:
positive_pair , negative_pair,output

# 4.3 create multiple models

In [ ]:
from tensorflow.keras import Model
training_model = Model(inputs=[image_input, caption_input, noise_input],outputs=output)
image_model = Model(inputs=image_input, outputs=image_pipeline) 
caption_model = Model(inputs=caption_input,outputs=caption_pipeline) 

# 5 Custom loss

In [ ]:
from keras import backend as K
def custom_loss(y_true, y_pred): positive = y_pred[:,0] negative = y_pred[:,1] 
    return K.sum(K.maximum(0., 1. - positive + negative)) 


In [ ]:
def accuracy(y_true, y_pred): positive = y_pred[:,0] negative = y_pred[:,1] 
    return K.mean(positive > negative)

In [ ]:
training_model.compile(loss=custom_loss, optimizer='adam',metrics=[accuracy])

# 6 Training

## 6.1 spiting the model

In [ ]:
noise = np.copy(captions)
fake_labels = np.zeros((len(features), 1))
X_train = ( [features[:9000], captions[:9000], noise[:9000]] ) 
Y_train = ( fake_labels[:9000])
X_valid = [features[-1000:], captions[-1000:], noise[-1000:]]
Y_valid = fake_labels[-1000:]

In [ ]:
#actual trainig 
for epoch in range(10): np.random.shuffle(noise) # don’t forget to shuffle mismatched captions
    training_model.fit(X_train, Y_train,validation_data=[X_valid, Y_valid],epochs =1,batch_size=64)    

## 6.2 saving model and representation 

In [ ]:
image_model.save('model.image') caption_model.save('model.caption') 
 
np.save('caption-representations',caption_model.predict(captions))
np.save('imagerepresentations', image_model.predict(feature))

# 7 Captioning novel images

In [ ]:
from keras.models import load_model
image_model = load_model('model.image',compile=False) 
caption_model = load_model('model.caption',compile=False)

## 7.1 load representation

In [ ]:
import numpy as np 
caption_representations = np.load('caption-representations.npy')
image_representations = np.load('image-representations.npy') 

In [ ]:
from keras.preprocessing.sequence import pad_sequences
import json
vocab = json.loads(open('vocab.json').read())
def preprocess_texts(texts): output = [] 
    for text in texts: output.append([vocab[word]
         if word in vocab 
            else 0 for word in text.split()])
    return pad_sequences(output, maxlen=16) 

In [ ]:
def generate_caption(image_filename, n=10):
    # generate image representation 
    for new image image_representation = image_model.predict(extract_features(image_filename)) # compute score of all captions in the dataset
    scores = np.dot(caption_representations,image_representation.T).flatten() # compute indices of n best captions 
    indices = np.argpartition(scores, -n)[-n:] 
    indices = indices[np.argsort(scores[indices])] # display them 
    for i in [int(x) for x in reversed(indices)]:
        print(scores[i], texts[i]) 

# 8 searching for images 

In [ ]:
def search_image(caption, n=10):
    caption_representation = caption_model.predict(preprocess_texts([caption]))
    scores = np.dot(image_representations, caption_representation.T).flatten() 
    indices = np.argpartition(scores, -n)[-n:] 
    indices = indices[np.argsort(scores[indices])] 
    for i in [int(x) for x in reversed(indices)]:
        print(scores[i], images[i]) 

In [ ]:
search_image('a man in the snow on some skis')